In [1]:
def path_cost(path : list, weights : dict, cycle=True) -> int:
    """Computes the cost of the given path

    Args:
        path (list): tour taken\n
        weights (dict): key -> edge : value -> weight.\n
        cycle (bool, optional): [description]. Defaults to True.\n

    Returns:
        int: final cost
    """
    cost = 0
    for i in range(len(path) - 1):
        try:
            if (path[i], path[i + 1]) in weights.keys():
                cost += weights[(path[i],path[i+1])]
            else:
                cost += weights[(path[i + 1],path[i])]
        except:
            if cycle:
                if (path[i], path[0]) in weights.keys():
                    cost += weights[(path[i], path[0])]
                else:
                    cost += weights[(path[0], path[i])]
                pass
    return cost


In [2]:
import math
def compute_distance(a, b):
    a = a['pos']
    b = b['pos']
    return math.sqrt(math.pow(a[0]-b[0],2) + math.pow(a[1]-b[1],2))


In [3]:
from graph import MyGraph
import torch
import os

def create_graph_fittizio(
     name : str,
     dir : str,
     lb : int = 0,
     ub : int = 100,
     n_nodes : int = 50,
     #normalized : bool = False,
     count : int = 0
    ):
    G = nx.Graph()
    V = set([v + 1 for v in range(n_nodes)])
    
    E = set()
    for combination in combinations(V, 2):
        E.add(combination)
    coords = []
    to_normalize_location = []
    max_coord = 0
    for i in range(1,n_nodes + 1):
        location = (random.randint(lb,ub), random.randint(lb,ub))
        if location[0] > max_coord:
            max_coord = location[0]
        elif location[1] > max_coord:
            max_coord = location[1]
        to_normalize_location.append(location)
    #print(to_normalize_location)
    for i, location in enumerate(to_normalize_location):
        normalized_location = (location[0]/max_coord, location[1]/max_coord)
        #print(normalized_location)
        coords.append(normalized_location)
        G.add_node(i+1, pos = normalized_location)
    coords = torch.Tensor(coords)
    for edge in E:
        w = compute_distance(G.nodes[edge[0]], G.nodes[edge[1]])
        #print(edge[0], edge[1], w)
        G.add_edge(edge[0], edge[1], weight = w)
    
    tsp = nx.approximation.traveling_salesman_problem
    path = tsp(G, cycle=True)
    if len(path) > 51:
        return count
    path = np.array(path)
    w_dic = {}
    for i in range(1, 51):
        for j in range(1,51):
            if i != j:
                w_dic[(i,j)] = compute_distance(G.nodes[i], G.nodes[j])
    #print(w_dic)
    cost = path_cost(path, weights = w_dic)
    #print(path, cost)
    

    #TODO salvare i grafi creati
    g = MyGraph(name + '.mio', nodes = G.nodes(), coords = coords,weights = w_dic, sub_opt = path, sub_opt_cost= cost)
    #print(len(path))
    path_to_save = os.path.join(dir, name)
    torch.save(g, path_to_save)
    return count + 1

In [4]:
import networkx as nx
import numpy as np
import random
from itertools import combinations

def get_weight(a, b):
    a = a['pos']
    b = b['pos']
    return int(np.sqrt((a[0]-b[0])**2 + (b[1]-a[1])**2))



In [5]:
from utility import create_dir
name_dir = 'generated_train'
create_dir(name_dir)
count = 0
while count < 100000:
    count = create_graph_fittizio(f'{name_dir}_' + str(count) + '.mio', name_dir, ub = 1000, count = count)

Path created correctly


In [6]:
for i in range(100000):
    x = torch.load(f'{name_dir}/{name_dir}_{i}.mio')
    if (len(x.sub_opt)) > 51:
        print(x.sub_opt_cost)
        print(x.coords)


In [7]:
from utility import create_dir
name_dir = 'generated_test'
create_dir(name_dir)
count = 0
while count < 10000:
    count = create_graph_fittizio(f'{name_dir}_' + str(count) + '.mio', name_dir, ub = 1000, count = count)

Path created correctly


In [ ]:
for i in range(10000):
    x = torch.load(f'{name_dir}/{name_dir}_{i}.mio')
    if (len(x.sub_opt)) != 51 or (x.sub_opt_cost > 8):
        print(i)
        print(x.coords.shape)
        print(x.coords)
        print(x.nodes)
